## Libraries

In [77]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights
from survey_dud_detector import detect_straightlining


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(for_vote, against_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 4.0
    ACQUIESENCE_BIAS = -5.0
    N_SIMS = 10000000
    
    for_moe = margin_of_error(n=n, p=for_vote/100, interval_size=interval)
    against_moe = margin_of_error(n=n, p=against_vote/100, interval_size=interval)
    undecided = 100 - for_vote - against_vote
    mean = for_vote + undecided * 0.25 + ACQUIESENCE_BIAS
    raw_moe = for_moe * 100 + against_moe * 100
    
    allocate_undecided = undecided * 0.4
    margin = raw_moe + allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    sigma = margin / 100 / normed_sigma
    
    sims = np.random.normal(mean / 100, sigma, N_SIMS)
    chance_pass = np.sum([s > 0.5 for s in sims]) / N_SIMS
    low, high = np.percentile(sims, [20, 80]) * 100
    
    return {'mean': mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': raw_moe, 'margin': margin, 'sigma': sigma, 'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < 0:
        first = 0
    print(('Result {} (80% CI: {} to {}) (N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} likely to pass)').format(mean,
                                                      first,
                                                      second,
                                                      n,
                                                      raw_moe,
                                                      margin,
                                                      sigma,
                                                      chance_pass))
    print(('{} (80% CI: {} to {}) ({})').format(mean,
                                                first,
                                                second,
                                                chance_pass))
    print('-')
    

## Load Processed Data

In [78]:
survey = pd.read_csv('responses_processed_national_weighted.csv').fillna('Not presented')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,78,79,80,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Candidate evaluations

## Insect Feed Attitudes

In [79]:
questions = {'agree_flies_animal_feed': 'How much do you agree or disagree with the following?: Flies are a suitable source of protein for use in animal feed',
            'agree_eat_animals_fed_insects': 'How much do you agree or disagree with the following?: I would eat meat if I knew it the animal had been fed insects as part of its feed',
            'agree_eat_insects': 'How much do you agree or disagree with the following?: I would never personally eat insects',
            'agree_flies_animal_feed_alt': 'How much do you agree or disagree with the following?: It is acceptable to feed insects to animals that are raised for food',
            'agree_eat_animals_fed_insects_alt_reverse': 'How much do you agree or disagree with the following?: I would not be comfortable eating meat from an animal that was raised on insect feed',
            'agree_insect_feed_harms_health': 'How much do you agree or disagree with the following?: Eating meat from an animal that was raised on feed made from insects poses a risk to human health'}

options = ['Agree', 'Disagree', 'Don\'t know']

for var, question in questions.items():
    print('## {} ##'.format(question))
    survey_ = survey.loc[survey[var].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    print(survey_[var].value_counts(normalize=True) * survey_.groupby(var)['weight'].mean() * 100)
    print('-')

## How much do you agree or disagree with the following?: Flies are a suitable source of protein for use in animal feed ##
Agree         24.288991
Disagree      34.214749
Don't know    41.496259
dtype: float64
-
## How much do you agree or disagree with the following?: I would eat meat if I knew it the animal had been fed insects as part of its feed ##
Agree         37.387022
Disagree      34.305828
Don't know    28.307151
dtype: float64
-
## How much do you agree or disagree with the following?: I would never personally eat insects ##
Agree         67.713651
Disagree      17.418804
Don't know    14.867545
dtype: float64
-
## How much do you agree or disagree with the following?: It is acceptable to feed insects to animals that are raised for food ##
Agree         37.297489
Disagree      30.490010
Don't know    32.212501
dtype: float64
-
## How much do you agree or disagree with the following?: I would not be comfortable eating meat from an animal that was raised on insect feed ##
Agre

In [80]:
straightlining_flies = detect_straightlining(survey[[c for c in survey.columns if 'agree_flies' in c or 'agree_eat' in c or 'agree_insects' in c]])
survey['meta_straightlining_flies'] = straightlining_flies
survey['meta_straightlining_flies'].value_counts()

0.6    1913
1.0    1107
0.4    1102
0.8     811
Name: meta_straightlining_flies, dtype: int64

In [81]:
pd.crosstab(survey['agree_flies_animal_feed'], survey['agree_flies_animal_feed_alt'])
# Flies are a suitable source of protein for use in animal feed
# It is acceptable to feed insects to animals that are raised for food

agree_flies_animal_feed_alt,Agree,Disagree,Don't know,Not presented
agree_flies_animal_feed,,,,
Agree,857,42,89,0
Disagree,181,660,251,0
Don't know,651,229,1115,0
Not presented,0,0,0,858


## Animal vote attitudes

In [82]:
options = ['Agree', 'Disagree', 'Don\'t know']

for c in survey.columns:
    if 'agree_vote' in c:
        print('## {} ##'.format(c))
        survey_ = survey.loc[survey[var].isin(options)].copy()
        survey_['weight'] = normalize_weights(survey_['weight'])
        print(survey_[c].value_counts(normalize=True) * survey_.groupby(c)['weight'].mean() * 100)
        print('-')
        
        

## agree_vote_atheist ##
Agree         15.910477
Disagree      50.421164
Don't know    33.668359
dtype: float64
-
## agree_vote_cand_animal_welfare ##
Agree         66.465326
Disagree      11.946448
Don't know    21.588227
dtype: float64
-
## agree_vote_cand_gun_control ##
Agree         52.069045
Disagree      32.677417
Don't know    15.253537
dtype: float64
-
## agree_vote_cand_oppose_abortion ##
Agree         39.613541
Disagree      39.396629
Don't know    20.989830
dtype: float64
-
## agree_vote_cand_reg_farming ##
Agree         52.406141
Disagree      13.158303
Don't know    34.435556
dtype: float64
-
## agree_vote_gay ##
Agree         15.736048
Disagree      40.723382
Don't know    43.540570
dtype: float64
-
## agree_vote_veg ##
Agree         14.876018
Disagree      43.579056
Don't know    41.544926
dtype: float64
-


In [ ]:
## Demographic support


In [48]:
print('## FeMALE ##')
options = ['Agree', 'Disagree', 'Don\'t know']

print('## Vegetarian candidate ##')
survey = survey[survey['gender'] == 'Female']
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['gender'] == 'Female']
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['gender'] == 'Female']
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('-')

print('-')
print('-')

## FeMALE ##
## Vegetarian candidate ##
## agree_vote_veg##
Agree         10.563332
Disagree      50.490093
Don't know    38.946575
dtype: float64
Weighted N = 764
## Farming regulation candidate ##
## agree_vote_cand_reg_farming##
Agree         50.759414
Disagree      13.423681
Don't know    35.816906
dtype: float64
Weighted N = 764
## Animal welfare candidate ##
## agree_vote_cand_animal_welfare ##
Agree         68.272142
Disagree       7.000394
Don't know    24.727464
dtype: float64
Weighted N = 764
-
-
-


In [56]:


print('## MALE ##')
options = ['Agree', 'Disagree', 'Don\'t know']

print('## Vegetarian candidate ##')
survey = survey[survey['gender'] == 'Male']
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['gender'] == 'Male']
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['gender'] == 'Male']
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('-')

print('-')
print('-')



## MALE ##
## Vegetarian candidate ##
## agree_vote_veg##
Agree         18.406156
Disagree      40.962244
Don't know    40.631600
dtype: float64
Weighted N = 713
## Farming regulation candidate ##
## agree_vote_cand_reg_farming##
Agree         50.482566
Disagree      15.679948
Don't know    33.837486
dtype: float64
Weighted N = 713
## Animal welfare candidate ##
## agree_vote_cand_animal_welfare ##
Agree         60.644414
Disagree      16.348035
Don't know    23.007551
dtype: float64
Weighted N = 713
-
-
-


In [ ]:
print('## FeMALE ##')
options = ['Agree', 'Disagree', 'Don\'t know']

print('## Vegetarian candidate ##')
survey = survey[survey['gender'] == 'Female']
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['gender'] == 'Female']
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['gender'] == 'Female']
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('-')

print('-')
print('-')

In [62]:


print('## BLACK OR AFRICAN AMERICAN ##')

options = ['Agree', 'Disagree', 'Don\'t know']


print('## Vegetarian candidate ##')
survey = survey[survey['race'] == 'Black or African American']
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['race'] == 'Black or African American']
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['race'] == 'Black or African American']
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))
print('-')
print('-')


## BLACK OR AFRICAN AMERICAN ##
## Vegetarian candidate ##
## agree_vote_veg##
Agree         23.469706
Disagree      45.605039
Don't know    30.925256
dtype: float64
Weighted N = 134
## Farming regulation candidate ##
## agree_vote_cand_reg_farming##
Agree         52.049179
Disagree      18.596664
Don't know    29.354157
dtype: float64
Weighted N = 134
## Animal welfare candidate ##
## agree_vote_cand_animal_welfare ##
Agree         62.445924
Disagree       8.686769
Don't know    28.867307
dtype: float64
Weighted N = 134
-
-


In [69]:

print('## WHITE OR CAUCASIAN ##')
options = ['Agree', 'Disagree', 'Don\'t know']


print('## Vegetarian candidate ##')
survey = survey[survey['race'] == 'White or Caucasian']
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['race'] == 'White or Caucasian']
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['race'] == 'White or Caucasian']
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))


print('-')
print('-')



## WHITE OR CAUCASIAN ##
## Vegetarian candidate ##
## agree_vote_veg##
Agree         14.501933
Disagree      48.225686
Don't know    37.272381
dtype: float64
Weighted N = 1053
## Farming regulation candidate ##
## agree_vote_cand_reg_farming##
Agree         53.385617
Disagree      13.531303
Don't know    33.083079
dtype: float64
Weighted N = 1053
## Animal welfare candidate ##
## agree_vote_cand_animal_welfare ##
Agree         68.378262
Disagree      10.398159
Don't know    21.223579
dtype: float64
Weighted N = 1053
-
-


In [76]:

print('## HISPANIC OR LATINO ##')

options = ['Agree', 'Disagree', 'Don\'t know']

print('## Vegetarian candidate ##')
survey = survey[survey['race'] == 'Hispanic or Latino']
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['race'] == 'Hispanic or Latino']
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['race'] == 'Hispanic or Latino']
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))
print('-')
print('-')



## HISPANIC OR LATINO ##
## Vegetarian candidate ##
## agree_vote_veg##
Agree          6.802097
Disagree      36.652094
Don't know    56.545809
dtype: float64
Weighted N = 109
## Farming regulation candidate ##
## agree_vote_cand_reg_farming##
Agree         34.736176
Disagree      18.041832
Don't know    47.221992
dtype: float64
Weighted N = 109
## Animal welfare candidate ##
## agree_vote_cand_animal_welfare ##
Agree         44.475838
Disagree      23.568763
Don't know    31.955399
dtype: float64
Weighted N = 109
-
-


In [83]:
print('## ASIAN OR ASIAN AMERICAN ##')

options = ['Agree', 'Disagree', 'Don\'t know']


print('## Vegetarian candidate ##')
survey = survey[survey['race'] == 'Asian or Asian American']
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['race'] == 'Asian or Asian American']
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['race'] == 'Asian or Asian American']
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))


print('-')
print('-')


## ASIAN OR ASIAN AMERICAN ##
## Vegetarian candidate ##
## agree_vote_veg##
Agree         12.531818
Disagree      31.747971
Don't know    55.720212
dtype: float64
Weighted N = 207
## Farming regulation candidate ##
## agree_vote_cand_reg_farming##
Agree         50.915270
Disagree       7.499254
Don't know    41.585476
dtype: float64
Weighted N = 207
## Animal welfare candidate ##
## agree_vote_cand_animal_welfare ##
Agree         69.797511
Disagree       3.112330
Don't know    27.090160
dtype: float64
Weighted N = 207
-
-


In [ ]:

print('## college ##')
options = ['Agree', 'Disagree', 'Don\'t know']

print('## Vegetarian candidate ##')
survey = survey[survey['college']]
survey_ = survey.loc[survey['agree_vote_veg'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_veg##')
print(survey_['agree_vote_veg'].value_counts(normalize=True) * survey_.groupby('agree_vote_veg')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Farming regulation candidate ##')
survey = survey[survey['college']]
survey_ = survey.loc[survey['agree_vote_cand_reg_farming'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_reg_farming##')
print(survey_['agree_vote_cand_reg_farming'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_reg_farming')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))

print('## Animal welfare candidate ##')
survey = survey[survey['college']]
survey_ = survey.loc[survey['agree_vote_cand_animal_welfare'].isin(options)].copy()
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
print('## agree_vote_cand_animal_welfare ##')
print(survey_['agree_vote_cand_animal_welfare'].value_counts(normalize=True) * survey_.groupby('agree_vote_cand_animal_welfare')['lv_weight_alt'].mean() * 100)
weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
print('Weighted N = {}'.format(weighted_n))


print('-')
print('-')

In [ ]:
    from scipy.stats import chi2_contingency
contigency= pd.crosstab(survey_['agree_vote_cand_animal_welfare'], survey_['agree_vote_cand_reg_farming']) 
contigency


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contigency) 
# Print the p-value
print(p)


In [ ]:
pd.crosstab(survey['agree_pain_ants'], survey['agree_flies_animal_feed'], normalize='index')